In [ ]:
%%bigquery
LOAD DATA INTO `qwiklabs-gcp-01-4b0fd1e23a0f.emergency_calls_response_times.emergency_calls_response_times_raw`
FROM FILES (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv']
)

Query is running:   0%|          |

""


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `emergency_calls_response_times.emergency_calls_response_times_model`
OPTIONS
  (model_type='linear_reg', DATA_SPLIT_METHOD='RANDOM', MAX_ITERATIONS=25)
  AS
SELECT
  response_time AS label, call_timestamp,	call_type,	location,	weather_condition,	day_of_week,	time_of_day,	traffic_level,	distance_to_station,	units_available
FROM `qwiklabs-gcp-01-4b0fd1e23a0f.emergency_calls_response_times.emergency_calls_response_times_raw`

Query is running:   0%|          |

""


In [1]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(
    MODEL `emergency_calls_response_times.emergency_calls_response_times_model`,
    (
      SELECT
        response_time AS label, call_timestamp, call_type, location, weather_condition,
        day_of_week, time_of_day, traffic_level, distance_to_station, units_available
      FROM
        `qwiklabs-gcp-01-4b0fd1e23a0f.emergency_calls_response_times.emergency_calls_response_times_raw`
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.739748,4.728384,0.014718,1.47647,0.831469,0.831472


In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.PREDICT(
    MODEL `emergency_calls_response_times.emergency_calls_response_times_model`,
    (
      SELECT
        TIMESTAMP("2023-01-15 21:18:33 UTC") AS call_timestamp,
        "Medical" AS call_type,
        "Uptown" AS location,
        "Sunny" AS weather_condition,
        "Sunday" AS day_of_week,
        21 AS time_of_day,
        "Low" AS traffic_level,
        0.55 AS distance_to_station,
        18 AS units_available
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_label,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,3.496993,2023-01-15 21:18:33+00:00,Medical,Uptown,Sunny,Sunday,21,Low,0.55,18
